In [1]:
import os
import polars as pl
from polars import col as c
import polars.selectors as cs
from datetime import timedelta
import tqdm
from datetime import datetime
import numpy as np
import json
from plotly.subplots import make_subplots
from data_federation.input_model import SmallflexInputSchema
import plotly.graph_objs as go
from polars_function import concat_list_of_list
from general_function import dictionary_key_filtering, pl_to_dict, generate_log, extract_archive, scan_folder
from numpy_function import relative_error_within_boundaries, error_within_boundaries
import plotly.express as px
from config import settings
from rpy2.robjects import pandas2ri
import rpy2.robjects as ro 
from pathlib import Path 
import random

from data_display.input_data_plots import plot_historical_scenarios

os.chdir(os.getcwd().replace("/src", ""))

COLORS = px.colors.qualitative.Plotly



2025-02-13 09:06:04 B01BPC15 rpy2.situation[33073] INFO cffi mode is CFFI_MODE.ANY
2025-02-13 09:06:04 B01BPC15 rpy2.situation[33073] INFO R home found: /usr/lib/R
2025-02-13 09:06:04 B01BPC15 rpy2.situation[33073] INFO R library path: /usr/lib/R/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/default-java/lib/server
2025-02-13 09:06:04 B01BPC15 rpy2.situation[33073] INFO LD_LIBRARY_PATH: /usr/lib/R/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/default-java/lib/server
2025-02-13 09:06:04 B01BPC15 rpy2.rinterface_lib.embedded[33073] INFO Default options to initialize R: rpy2, --quiet, --no-save
2025-02-13 09:06:04 B01BPC15 rpy2.rinterface_lib.embedded[33073] INFO R is already initialized. No need to initialize.


In [2]:
os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = "/home/ltomasini/gruobi_license/gurobi.lic"


In [3]:
output_file_names: dict[str, str] = json.load(open(settings.OUTPUT_FILE_NAMES)) # type: ignore
plot_folder = output_file_names["input_data_plot"] + "/historical_scenarios"

small_flex_input_schema: SmallflexInputSchema = SmallflexInputSchema()\
.duckdb_to_schema(file_path=output_file_names["duckdb_input"])

Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 16/16 [00:02<00:00,  6.92it/s]


In [ ]:
day_discharge_flow = small_flex_input_schema.discharge_flow_historical.with_columns(
    c("timestamp").dt.ordinal_day().alias("day"),
    c("timestamp").dt.year().alias("year")
).group_by("year", "day", "location").agg(c("value").sum()*3.6).sort("year", "day", "location")

week_discharge_flow = small_flex_input_schema.discharge_flow_historical.with_columns(
    c("timestamp").dt.week().alias("week"),
    c("timestamp").dt.year().alias("year")
).group_by("year", "week", "location").agg(c("value").sum()*3.6).sort("year", "week", "location")

plot_historical_scenarios(
    data=day_discharge_flow, title="Sum daily discharge flow [km3/s]", 
    x_axis="day", plot_folder=plot_folder, plot_name="daily_discharge_flow_historical")

plot_historical_scenarios(
    data=week_discharge_flow, title="Sum weekly discharge flow [km3/s]", 
    x_axis="week", plot_folder=plot_folder, plot_name="weekly_discharge_flow_historical")

In [ ]:
filter_mask = (
    ((c("sub_basin") == "Greisse_1") & (c("start_height") == 2750)) | 
    ((c("sub_basin") == "Greisse_2") & (c("start_height") == 2450)) |
    ((c("sub_basin") == "Greisse_3") & (c("start_height") == 2750)) |
    ((c("sub_basin") == "Greisse_4") & (c("start_height") == 2450))
)

weather_historical = small_flex_input_schema.weather_historical.filter(filter_mask).with_columns(
    c("timestamp").dt.ordinal_day().alias("day"),
    c("timestamp").dt.week().alias("week"),
    c("timestamp").dt.year().alias("year"),
    c("sub_basin").alias("location")
).filter(c("year") != 2005)

day_irradiation = weather_historical\
    .rename({"irradiation": "value"})\
    .group_by("year", "day", "location")\
    .agg(c("value").sum()*3.6).sort("year", "day", "location")


week_irradiation = weather_historical\
    .rename({"irradiation": "value"})\
    .group_by("year", "week", "location")\
    .agg(c("value").sum()*3.6).sort("year", "week", "location")
    
plot_historical_scenarios(
    data=day_irradiation, title="Sum daily irradiation [kWh/m2]", 
    x_axis="day", plot_folder=plot_folder, plot_name="daily_irradiation_historical")

plot_historical_scenarios(
    data=week_irradiation, title="Sum weekly irradiation [kWh/m2]", 
    x_axis="week", plot_folder=plot_folder, plot_name="weekly_irradiation_historical")

In [14]:
day_wind = weather_historical\
    .with_columns(c("wind").pow(3).alias("value"))\
    .group_by("year", "day", "location")\
    .agg(c("value").mean()).sort("year", "day", "location")


week_wind = weather_historical\
    .with_columns(c("wind").pow(3).alias("value"))\
    .group_by("year", "week", "location")\
    .agg(c("value").mean()).sort("year", "week", "location")
    
plot_historical_scenarios(
    data=day_wind, title="Mean daily wind [m^3/s^3]", 
    x_axis="day", plot_folder=plot_folder, plot_name="daily_wind_historical")

plot_historical_scenarios(
    data=week_wind, title="Mean weekly wind [m^3/s^3]", 
    x_axis="week", plot_folder=plot_folder, plot_name="weekly_wind_historical")

In [23]:
market_price

timestamp,market,direction,country,source,unit,max,avg,min,day,week,year
"datetime[μs, UTC]",str,str,str,str,str,f64,f64,f64,i16,i8,i32
2015-01-01 00:00:00 UTC,"""DA""","""sym""","""CH""","""alpiq""","""EUR/MWh""",null,44.94,null,1,1,2015
2015-01-01 01:00:00 UTC,"""DA""","""sym""","""CH""","""alpiq""","""EUR/MWh""",null,43.43,null,1,1,2015
2015-01-01 02:00:00 UTC,"""DA""","""sym""","""CH""","""alpiq""","""EUR/MWh""",null,38.08,null,1,1,2015
2015-01-01 03:00:00 UTC,"""DA""","""sym""","""CH""","""alpiq""","""EUR/MWh""",null,35.47,null,1,1,2015
2015-01-01 04:00:00 UTC,"""DA""","""sym""","""CH""","""alpiq""","""EUR/MWh""",null,30.83,null,1,1,2015
…,…,…,…,…,…,…,…,…,…,…,…
2023-12-31 19:00:00 UTC,"""DA""","""sym""","""IT-NORD""","""alpiq""","""EUR/MWh""",null,136.07,null,365,52,2023
2023-12-31 20:00:00 UTC,"""DA""","""sym""","""IT-NORD""","""alpiq""","""EUR/MWh""",null,130.0,null,365,52,2023
2023-12-31 21:00:00 UTC,"""DA""","""sym""","""IT-NORD""","""alpiq""","""EUR/MWh""",null,117.23116,null,365,52,2023


In [25]:
market_price = small_flex_input_schema.market_price_measurement.filter(c("market") == "DA").with_columns(
    c("timestamp").dt.ordinal_day().alias("day"),
    c("timestamp").dt.week().alias("week"),
    c("timestamp").dt.year().alias("year"),
).rename({"avg": "value", "country": "location"})\

day_market_price = market_price\
    .group_by("year", "day", "location")\
    .agg(c("value").mean()).sort("year", "day", "location")


week_market_price = market_price\
    .group_by("year", "week", "location")\
    .agg(c("value").mean()).sort("year", "week", "location")
    
plot_historical_scenarios(
    data=day_market_price, title="Mean daily wind [m^3/s^3]", 
    x_axis="day", plot_folder=plot_folder, plot_name="daily_market_price_historical")

plot_historical_scenarios(
    data=week_market_price, title="Mean weekly wind [m^3/s^3]", 
    x_axis="week", plot_folder=plot_folder, plot_name="weekly_market_price_historical")